### Setup Langchain + LLM
1. Install Langchain: 
- pip intall langchain
2. Install integration packages.
- pip install -U langchain-cohere
- pip install -U langchain-groq
- pip install -U langchain-mistralai

In [2]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('/workspaces/langchainCDAC9nov/config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

messages = [
    SystemMessage(content='You are a weather service. You will respond to weather queries to the best of you ability. You will always end with - Have a great day'),
    HumanMessage(content='Hey whats the weather like today?')
]



## code for cohere.
model = ChatCohere(model="command-r-plus")
print(model.invoke(messages))

## Code for Groq
model = ChatGroq(model="llama3-8b-8192")
print(model.invoke(messages))



content='Hi there! I can help you with that. \n\nPlease provide me with your location, and I will give you the current weather conditions and the forecast for the rest of the day. \n\n- Have a great day!' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1fcfcc9d-7440-4148-a424-eddeab77e31d', 'token_count': {'input_tokens': 237.0, 'output_tokens': 46.0}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1fcfcc9d-7440-4148-a424-eddeab77e31d', 'token_count': {'input_tokens': 237.0, 'output_tokens': 46.0}} id='run-5517e0f5-9206-4440-96f6-08f32edd2daf-0' usage_metadata={'input_tokens': 237, 'output_tokens': 46, 'total_tokens': 283}
content="I'd be happy to help you with that! As of now, the current weather conditions are quite pleasant. There's a gentle breeze blowing, and the temperature is

In [3]:
messages = [
    SystemMessage(content='You are a Capital_City service. You will tell the Capital_City of the State. Have a great day!!'),
    HumanMessage(content='Delhi is Capital of which Country')
]
# model = ChatCohere(model="command-r-plus")
# print(model.invoke(messages))

# Code for Groq
model = ChatGroq(model="llama3-8b-8192")
print(model.invoke(messages))


content='Delhi is the capital of India.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 46, 'total_tokens': 55, 'completion_time': 0.0075, 'prompt_time': 0.008492568, 'queue_time': 0.006163749999999999, 'total_time': 0.015992568}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None} id='run-c41994e3-1352-433b-b151-159898307055-0' usage_metadata={'input_tokens': 46, 'output_tokens': 9, 'total_tokens': 55}


## Create the prompt
1. Imports Human and System message classes. System represents our instructions to GPT and Human represents the question or prompt that the user provides.
2. LangChain responses are instances of class `BaseMessage` It contains the actual response from GPT and some other metadata.
3. Since we are interested only in the string reponse that GPT gave we chain (pipe) the reponse to a parser
4. For our purpose we will use `StrOutputParser` class
5. Next we create a `chain` using the components `model` and `parser`
6. Finally we call the `invoke` method on the chain and pass our `messages` list to it.
7. In the output cell we get the response from `GPT-35-turbo`

*A chain is an wrapper around multiple individual components that are executed in a defined order. Components in LangChain implement `Runnable` interface. This interface have a method `invoke` that transforms single input to an output.*


In [4]:
#The classes used for setting up the prompt
import puzzles
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()

puzzle = puzzles.puzzles('hungryLions') # Based on user input pick a puzzle.

# templatized system prompt
system_template = "solve the following puzzle. Please provide a {responseType} response." 

# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", puzzle)
    ]
)


# prompt Template also implements runnable and can be easily chained.
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"100 words"})

client=<cohere.client.Client object at 0x7ad2e8a910d0> async_client=<cohere.client.AsyncClient object at 0x7ad2e8a918e0> model='command-r' cohere_api_key=SecretStr('**********')
['/usr/local/python/3.12.1/lib/python312.zip', '/usr/local/python/3.12.1/lib/python3.12', '/usr/local/python/3.12.1/lib/python3.12/lib-dynload', '', '/home/codespace/.local/lib/python3.12/site-packages', '/usr/local/python/3.12.1/lib/python3.12/site-packages']


"This classic puzzle is a great example of lateral thinking! The man should choose the third room, full of tigers that haven't eaten for six months. Why? Because tigers that haven't eaten for six months are likely to be dead or very weak, making them no threat to the man's life. The firing squad and blazing fire are much more likely to result in his death, so the third room is the safest choice."

In [6]:
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()

# puzzle = puzzles.puzzles('hungryLions') # Based on user input pick a puzzle.
query=input('Enter the city for which you want the tourist places:  ')

# templatized system prompt
system_template = "return a output in json form as the tourist place in city and its ranking according to reviews as per the input city. Please provide a {size} related response." 

# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", query)
    ]
)


# prompt Template also implements runnable and can be easily chained.
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief","size":input("Enter the size you expect")})

{'Pune': [{'name': 'Agakhan Palace', 'ranking': 4.8, 'reviews': 1234},
  {'name': 'Shani Shingnapur', 'ranking': 4.6, 'reviews': 987},
  {'name': 'Osho Teerth Park', 'ranking': 4.5, 'reviews': 765},
  {'name': 'Pune Okayama Friendship Garden', 'ranking': 4.4, 'reviews': 543},
  {'name': 'Bhaja Caves', 'ranking': 4.3, 'reviews': 421}]}

### Chatbot 
1. We begin with creating a basic chatbot.

In [8]:
model = ChatGroq(model="llama3-8b-8192")
parser = StrOutputParser()
chain = model | parser

response = chain.invoke([HumanMessage(content="hi I am Bob")])

print(response)

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


#### Lets dig into what is happening here.
1. Click here to check the UML diagram: 
2. https://medium.com/azure-monitor-from-a-programmers-perspective/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#56cf


#### Runnable
1. Its an extremely prominent class and used extensively in creating chains.
2. Chains combine components together in a pipeline
3. Many components like all models, parsers, prompts and anything that can logically go into a chain derives from it.
4. `ChatGroq` is provided partner by extends `BaseChatModel` from langchain_core
5. https://github.com/langchain-ai/langchain/blob/master/libs/partners/groq/langchain_groq/chat_models.py
6. This is the base class for all model classes offered by any partner.
7. `BaseClass` extends `RunnableSerializable` that supports serialization into JSON
8. `RunnableSerializable` extends `Runnable` that means it can participate in chains.
9. You can also use `RunnableSequence` to construct the chain.
10. https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/runnables/base.py#L2659

In [ ]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(model, parser)
chain.invoke([HumanMessage(content="hi i am bob")])


"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [ ]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(model,parser)
chain.invoke([HumanMessage(content="Hii I am Atharva")])

"Nice to meet you, Atharva! How's your day going so far?"

1. Chain calls the first component and passes any arguments provided to it.
2. In this case its an object of type `HumanMessage`
3. This is how a chain looks: https://miro.medium.com/v2/resize:fit:750/format:webp/1*K1F-m4gImEUO0AELkpQuKg.jpeg
4. Each model component by any partner provides an object of type `BaseMessage`. This is then passed to the next component.
5. This is the signature of invoke of a model class

`def` `invoke(str | List[dict | tuple | BaseMessage] | PromptValue):`\
    Suite
  
6. In our example `HumanMessage` is derived from `BaseMessage` which needs `content` for initialization.

`param content: Union[str, List[Union[str,Dict]]]`

7. Union, List, Dict are all defined in typing module
8. Union means one of the input types is expected. We are passing a string.

9. Our `parser` is of type `StrOutputParser` that extends `BaseOutputParser`
10. Its invoke is:

`def invoke(self, input: Union[str, BaseMessage], config: Optional[RunnableConfig] = None) -> T:`

11.  This says input can be either string or `BaseMessage`. We are using `BaseMessage` the return type of `model`

12. Some useful methods are:
- parser.input_schema.schema() # get JSON schema of the input
- parser.output_schema.schema() # gets JSON schema of the output


### Adding history to chat
1. At this stage if you pass another message to the model it will have no recollection of the earlier message.
2. Lets add history. Chat history is managed by a set of classes offered by community.
3. https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/chat_history.py
4. `asyncio` is a Python library: https://docs.python.org/3/library/asyncio.html 

In [ ]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Now lets first add the first message to the store
store.add_message(HumanMessage('Hi! I am Bob'))

messages = await(getMessage())


response = model.invoke(messages) # asyncio has runners for coroutines, context managers etc. 
print(response.content) # note that our first message is safely in the store

# lets add the message returned by the model to the store

store.add_message(SystemMessage(response.content))


store.add_message(HumanMessage('Lets see if you know my name dude?'))

messages = await(getMessage())

print(messages) # check all the message are in store.

response = model.invoke(messages)

print(response.content) # Notice that the reponse now takes into account earlier interactions also.

Messages retrieved from DB
Hi Bob! Nice to meet you! Is there something I can help you with or would you like to chat?
Messages retrieved from DB
[HumanMessage(content='Hi! I am Bob', additional_kwargs={}, response_metadata={}), SystemMessage(content='Hi Bob! Nice to meet you! Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Lets see if you know my name dude?', additional_kwargs={}, response_metadata={})]
I knew it! Your name is Bob, right?


In [ ]:
# model2 = ChatGroq(model="llama3-8b-8192")
from langchain_core.chat_history import(BaseChatMessageHistory,InMemoryChatMessageHistory,)
store=InMemoryChatMessageHistory()
import asyncio
async def getMessage():
    await asyncio.sleep(2)
    print("Messages retrieved")
    return await store.aget_messages()
store.add_message(HumanMessage("I am Atharva and I am from Maharashtra"))
messages = await(getMessage())
response=model.invoke(messages)
print(response.content)
store.add_message(SystemMessage(response.content))


store.add_message(HumanMessage('Lets see if you can tell capital city of my state?'))

messages = await(getMessage())
print(messages)
response = model.invoke(messages)

print(response.content) 

Messages retrieved


Namaste Atharva! It's great to meet you. Maharashtra is a beautiful state with a rich culture and history. What brings you here today? Do you have any interests or hobbies you'd like to share?
Messages retrieved
[HumanMessage(content='I am Atharva and I am from Maharashtra', additional_kwargs={}, response_metadata={}), SystemMessage(content="Namaste Atharva! It's great to meet you. Maharashtra is a beautiful state with a rich culture and history. What brings you here today? Do you have any interests or hobbies you'd like to share?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Lets see if you can tell capital city of my state?', additional_kwargs={}, response_metadata={})]
That's a great question! I think the capital city of Maharashtra is... Mumbai! Am I correct?


1. There are some issues here. Since Chat History is not a descendant of Runnable we cannot chain it.
2. Therefore the code is sort of littered. 
3. Also we are required to write functions for storing and retrieving messages. This should be rather standard and done by the framework!
4. What about sessions? This code is running of the server which supports multiple users. So there needs to be a mechanism to manage sessions.

#### RunnableWithMessageHistory
1. This is where LangChain offers this class.
2. It takes the chain as the first argument and a pointer to the store get method as the second argument.
3. This class then takes the ownership of executing the chain and any component that 

In [1]:
# Lets create our own store. This store will be a dict with a key for each session
# The value for each key will be InMemoryChatHistory object 

from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
config = {'configurable': {"session_id": "abc2"}}
withHistory = RunnableWithMessageHistory(model, get_session_history)

response = withHistory.invoke([HumanMessage(content="Hi! I am Bob")], config=config)

print(response.content) # all good so far

# we dont need to explicitly store the response from the model in history

response = withHistory.invoke(
    [HumanMessage(content="Lets see if you know my name dude?")], config=config
)

print(response.content)

NameError: name 'BaseChatMessageHistory' is not defined

In [39]:

from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
config = {'configurable': {"session_id": "Sesssion-1"}}
newHistory = RunnableWithMessageHistory(model, get_session_history)

response = newHistory.invoke([HumanMessage(content="Hello I am Atharva and I am living in Pune")], config=config)

print(response.content) # all good so far

# we dont need to explicitly store the response from the model in history

response = withHistory.invoke(
    [HumanMessage(content="Do you remember my name?")], config=config
)

print(response.content)

Nice to meet you, Atharva! Pune is a great city, isn't it? What do you like most about living in Pune?
I remember that your name is Atharva, and you're living in Pune.


In [ ]:


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
config = {'configurable': {"session_id": "Session-2"}}
newHistory = RunnableWithMessageHistory(model, get_session_history)

response = newHistory.invoke([HumanMessage(content="Hello I am Rajan and I am living in Mumbai")], config=config)

print(response.content) # all good so far

# we dont need to explicitly store the response from the model in history

response = withHistory.invoke(
    [HumanMessage(content="Do you remember where do I belong ?")], config=config
)

print(response.content)

I remember! You're Rajan, and you're living in Mumbai!
I remember again! You said you belong in Mumbai!


In [42]:
print(store.keys())

dict_keys(['Sesssion-1', 'Session-2'])


In [43]:
print(store)

{'Sesssion-1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello I am Atharva and I am living in Pune', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Atharva! Pune is a great city, isn't it? What do you like most about living in Pune?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 22, 'total_tokens': 52, 'completion_time': 0.025, 'prompt_time': 0.000784625, 'queue_time': 0.012611942999999999, 'total_time': 0.025784625}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac94c00e-5e52-43bd-84e8-7f71738aa45e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 30, 'total_tokens': 52}), HumanMessage(content='Do you remember my name?', additional_kwargs={}, response_metadata={}), AIMessage(content="I remember that your name is Atharva, and you're living in Pune.", additional_kwargs={}, response_metadata={'token_

1. Here is a flowchart of this program.
2. https://medium.com/azure-monitor-from-a-programmers-perspective/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#3c92
3. Wrapper around another runnable - the chain
4. https://techblogs.cloudlex.com/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#a0cb